In [ ]:
!hostname

In [ ]:
import os
os.listdir()
os.chdir("/home/kaborg15/FREja")

In [3]:
import json
import Database_connector
import pandas as pd
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import torch
import numpy as np
from Database_Settings import DB_SETTINGS
import Database_connector
from mysql.connector import (connection)
from PIL import Image

PICKLED_DATA_FOLDER = "Pickled_Data/"
PICKLED_PICTURES_FILE = "pictures_embeddings_resnet152_df.pkl"
PICKLED_OUTFIRS_FILE = "active_outfits_df.pkl"

PICTURES_DROP_COLUMNS = ["contentType", "status", "displayOrder", "sourceURL", "embeddings"]
#PICTURES_DROP_COLUMNS = []
OUTFITS_DROP_COLUMNS = ['owner', 'name', 'brand', 'isPublic', 'isDeleted', 'meta.validFrom', 'meta.validTo', "Outfit_size"]


# def get_active_outfits():
#     cnx = connection.MySQLConnection(**DB_SETTINGS)
#     db_connection = Database_connector.Db_Connection()
#     cursor = cnx.cursor(dictionary=True)
#     #tag_query = "SELECT * FROM Outfits WHERE (`Outfits`.`isPublic` = TRUE AND `Outfits`.`isDeleted` = FALSE AND `Outfits`.`meta.validTo` >= '9999-01-01 00:00:00')"
#     tag_query = "SELECT `Outfits`.`id` AS `id`, `Outfits`.`owner` AS `owner`, `Outfits`.`name` AS `name`, `Outfits`.`brand` AS `brand`, `Outfits`.`isPublic` AS `isPublic`, `Outfits`.`isDeleted` AS `isDeleted`, `Outfits`.`timeCreated` AS `timeCreated`, `Outfits`.`timeUpdated` AS `timeUpdated`, `Outfits`.`pricePerWeek` AS `pricePerWeek`, `Outfits`.`pricePerMonth` AS `pricePerMonth`, `Outfits`.`type` AS `type`, `Outfits`.`keywords` AS `keywords`, `Outfits`.`retailPrice` AS `retailPrice`, `Outfits`.`meta.validFrom` AS `meta.validFrom`, `Outfits`.`meta.validTo` AS `meta.validTo` FROM `Outfits` WHERE (`Outfits`.`isPublic` = TRUE AND `Outfits`.`isDeleted` = FALSE AND `Outfits`.`meta.validTo` >= '9999-01-01 00:00:00')"
#     cursor.execute(tag_query)
#     outfit_results = cursor.fetchall()

#     outfits_df_db = pd.DataFrame([list(order_dict.values()) for order_dict in outfit_results], columns=list(outfit_results[0].keys()))
#     return outfits_df_db

def get_pictures_df(path):
    pictures_df = pd.read_pickle(path)
    #pictures_df = pictures_df.drop(columns=PICTURES_DROP_COLUMNS)
    pictures_df = pictures_df.drop(columns=pictures_df.columns[2:])
    return pictures_df

def get_outfits_df(path):
    outfits_df = pd.read_pickle(path)
    outfits_df = outfits_df.drop(columns=OUTFITS_DROP_COLUMNS)
    return outfits_df

def prepare_data(pictures_df_path = PICKLED_DATA_FOLDER + PICKLED_PICTURES_FILE, outfits_df_path = PICKLED_DATA_FOLDER + PICKLED_OUTFIRS_FILE):
    pictures_df = get_pictures_df(pictures_df_path)
    outfits_df = get_outfits_df(outfits_df_path)
    return pictures_df, outfits_df

def picture_exists(picture_id, pictures_dir_path):
    return os.path.isfile(pictures_dir_path + os.sep + picture_id + ".jpg")

def find_missing_pictures(pictures_df, pictures_dir_path):
    pictures_df["file_exists"] = pictures_df["id"].apply(lambda x : picture_exists(x, pictures_dir_path))
    return pictures_df["file_exists"].value_counts()

In [4]:
! pip install mysql-connector-python

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
pictures_df, outfits_df = prepare_data(pictures_df_path="../FREja_dataset_processing/pictures_embeddings_resnet152_df.pkl"
                                       , outfits_df_path="../FREja_dataset_processing/active_outfits_df.pkl")
pictures_df.head()

,id,owner
0,picture.00058abb53434872ae9bb4270ae21f8e,outfit.98f32aaf08bc4ff09c44e6e11e9199bc
1,picture.000cf715019a4a02aaf0dc479212195a,outfit.f11bdc25d87946f8b831b608bc8fb574
2,picture.0010c2e161154d6893734981d5455e76,outfit.9387d05b47f906c5
3,picture.00135ce8b1a04c5daa60cf7bdd99bcd5,outfit.12ae28ab0dc5494c98a1de2f8ce04b79
4,picture.001a58ea68da426384567b8cccc0c8a6,outfit.e989b8cb4a814d97b642e1cb326f47e6


In [6]:
PICTURES_DIR_PATH = "../FREja_dataset_processing/Filtered_pictures/"
DF_SPLITS = 200

DEVICE = "cuda:1"
EMBEDDINGS_SAVE_DIR = "Pickled_Data/Embeddings_resnet_layers/"

if not os.path.isdir(EMBEDDINGS_SAVE_DIR):
    os.mkdir(EMBEDDINGS_SAVE_DIR)

In [7]:
find_missing_pictures(pictures_df, PICTURES_DIR_PATH)

True    18060
Name: file_exists, dtype: int64

In [8]:
from torchvision import transforms
import copy
from IPython.display import display
import ipywidgets
from tqdm.notebook import tqdm
from torchvision.io import read_image
from torchvision.io import read_image

def load_images_torchvision(pictures_df, n):
    image_ids = pictures_df["id"][:n]
    images = []
    for image_id in image_ids:
        image_path = PICTURES_DIR_PATH + image_id + ".jpg"
        pil_image = read_image(image_path)
        images.append(pil_image)
    return images

def load_image_torchvision(picture_id):
    image_path = PICTURES_DIR_PATH + picture_id + ".jpg"
    image = read_image(image_path)
    return image

def load_image(image_id):
    file_name = image_id + ".jpg"
    file_path = PICTURES_DIR_PATH + file_name
    image = read_image(file_path)
    return image

def image_to_embedding(model_e, image_id, d_widget, index_num):
    d_widget.value = f"Converting file number: {index_num}"
    pil_image = load_image(image_id)
    #input_tensor = preprocess(pil_image)
    input_tensor = pil_image
    # if torch.cuda.is_available():
    #     input_tensor = input_tensor.to('cuda')

    embedding = model_e(input_tensor)
    return embedding[0].squeeze().cpu()

def get_df_embeddings(pictures_df, embedding_model, preprocess):
    display_out = ipywidgets.HTML()
    display(display_out)

    if torch.cuda.is_available():
        embedding_model.to('cuda')

    split_dfs = np.array_split(pictures_df, DF_SPLITS)
    with torch.no_grad():
        for df_split in tqdm(split_dfs):
            df_split["embeddings"] = df_split.apply(lambda row: image_to_embedding(preprocess, embedding_model, row["id"], display_out, row.name), axis=1)
    embedding_df = pd.concat(split_dfs)
    return embedding_df


In [9]:
from tqdm import tqdm, tqdm_notebook
from torchvision.models import quantization
from torchvision import models
import torchvision

def load_image_series_torchvision(image_series):
    images = []
    for image_id in image_series:
        image_path = PICTURES_DIR_PATH + image_id + ".jpg"
        torch_image = read_image(image_path, mode=torchvision.io.ImageReadMode.RGB)
        images.append(torch_image)
    return images

class Embedding_Config():
    class exception_hack(Exception):
        pass

    
    class Parasite_Module(torch.nn.Module):
        def __init__(self, host_layer, include_host=True):
            super().__init__()
            self.host_layer = host_layer
            self.include_host = include_host

        def forward(self, x):
            if self.include_host:
                x = self.host_layer(x)
            raise Embedding_Config.exception_hack(x)

    def __init__(self):
        self.use_cuda = None
        
        self.weights = None
        self.transforms = None
        self.model = None
        self.model_save_name = None

    def load_model(self):
        raise NotImplementedError

    def prepare_model(self):
        model = self.load_model()
        if self.use_cuda:
            model.to(DEVICE)
        model.eval()
        return model

    def print_config_summary(self):
        print(f"PKL name: {self.model_save_name} | Model: {type(self.model).__name__} | Weights: {type(self.weights).__name__} | {self.weights.name}")

    def load_images(self, image_series):
        image_tensors = load_image_series_torchvision(image_series)
        formatted_images = [self.transforms(test_img).unsqueeze(0) for test_img in image_tensors]
        formatted_images = torch.vstack(formatted_images)
        if self.use_cuda:
            formatted_images = formatted_images.to(DEVICE)
        return formatted_images
    
    def get_embeddings(self, image_tensors):
        with torch.no_grad():
            try:
                embeddings = self.model(image_tensors)
            except self.exception_hack as e:
                embeddings = e.args[0]
        # Reduce to half precision to save space and load time
        forward_embeddings = embeddings.cpu()#embeddings.half().cpu()
        if 1 in forward_embeddings.shape:
            print(f"Squeezing tensor with shape: {forward_embeddings.shape}")
            forward_embeddings = forward_embeddings.squeeze()
        return forward_embeddings.half()

class resnet50_V1_Output(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class resnet50_V1_Final_Layer(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class resnet50_V1_avgpool_in(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.avgpool = self.Parasite_Module(model.avgpool, include_host=False)
        return model

class resnet50_V1_layer4_2_relu(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.layer4[2].relu = self.Parasite_Module(model.layer4[2].relu, include_host=True)
        return model

class resnet50_V1_layer4_2_conv1(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.layer4[2].conv1 = self.Parasite_Module(model.layer4[2].conv1, include_host=False)
        return model

class resnet50_V1_layer4_2_conv3(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.layer4[2].conv3 = self.Parasite_Module(model.layer4[2].conv3, include_host=True)
        return model

class resnet50_V1_layer4_2_conv2(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.layer4[2].conv2 = self.Parasite_Module(model.layer4[2].conv2, include_host=True)
        return model

class resnet50_V2_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

# for test_class in [resnet50_V1_layer4_2_conv1, resnet50_V1_layer4_2_conv2, resnet50_V1_layer4_2_conv3]:
#     test_config_class = test_class()
#     test_loaded_images = test_config_class.load_images(pictures_df["id"][:10])
#     test_embeddings = test_config_class.get_embeddings(test_loaded_images)
#     print(test_loaded_images.shape, test_embeddings.shape)

In [10]:
def get_df_embeddings(base_df, config_class : Embedding_Config):
    model_config = config_class()
    print(f"Generating embeddings from {type(model_config).__name__} to {model_config.model_save_name}...")
    split_dfs = np.array_split(base_df, DF_SPLITS)
    for df_split in tqdm(split_dfs):
        df_images = model_config.load_images(df_split["id"])
        df_embeddings = list(model_config.get_embeddings(df_images))
        df_split["embeddings"] = df_embeddings
    embedding_df = pd.concat(split_dfs)
    embedding_df.to_pickle(os.path.join(EMBEDDINGS_SAVE_DIR, model_config.model_save_name))
    return embedding_df

class_configs = [resnet50_V1_layer4_2_conv1]
for config_class in class_configs:
    get_df_embeddings(pictures_df, config_class)

PKL name: resnet50_V1_layer4_2_conv1.pkl | Model: ResNet | Weights: ResNet50_Weights | IMAGENET1K_V1
Generating embeddings from resnet50_V1_layer4_2_conv1 to resnet50_V1_layer4_2_conv1.pkl...


100%|██████████| 200/200 [30:14<00:00,  9.07s/it]


OSError: [Errno 28] No space left on device